In [1]:
import sys
import zipfile
import os

import numpy as np
import pandas as pd
from IPython.display import display, HTML

sys.path.append("../../libs/")

from datasetutil import gerar_estatiscas_df, substituir_valores_nulos , verify_gg4cc , save_zip

In [7]:

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Metodos

In [36]:
def read(zinput,nome_arquivo,
         file_format=".csv",
         separador=',',codificacao="iso-8859-1",quote='True',skip_coments=False):
    '''
        Ler zip com os datasets
    '''
    zf = zipfile.ZipFile(zinput) 

    for file in zipfile.ZipFile.namelist(zf):
        if file == nome_arquivo+file_format:
            if quote == 'True':
                df = pd.read_csv(zf.open(file), header=0, encoding=codificacao,
                                 sep=separador,error_bad_lines=False)#,
            else:
                df = pd.read_csv(zf.open(file), header=0, encoding=codificacao,
                                 sep=separador,error_bad_lines=False,
                                 quotechar = quote)
            if skip_coments:
                df = pd.read_csv(zf.open(file), header=0, encoding=codificacao,
                                 sep=separador,error_bad_lines=False,comment='#')
                
#             chunksize=640000000,
#             nrows=15)
            return df
# df.iloc[:, [68,2,10,11,9,28,29,13,14,15,1,16]]

In [124]:
def procurar_duplicados(_df,a='id'):
    counts = _df[a].value_counts()
    return _df[_df[a].isin(counts.index[counts > 1])]

In [125]:
def retornar_ds_limpo(_df,chave='id'):
    saida = []
    
    for k in _df[chave].unique():
        if len(saida) == 0:
            saida = _df[_df[chave] == k].head(1)
        else:
            saida = pd.concat([saida,_df[_df[chave] == k].head(1)])
    return saida

# Magellan

## restaurantes

### restaurantes4

In [121]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Magellan' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'restaurants'+os.sep

inputZip = 'yelp_yellowpages.zip'
data = 'labeled_data'


zip_file = 'processed_yelp-yellowpages.zip'

df = read(ds_files+inputZip,data,skip_coments=True)

In [122]:
# gabarito
gs = df[df.gold==1].iloc[:,[1,2]]


colunas =  [ 'id','address','city','name','phone','state','zipcode' ]
n_atibutos = len(colunas) -1

inicio_atributos_ds1 = 3
fim_atributos_ds1 = inicio_atributos_ds1 + n_atibutos


atts = [1] + list(range(inicio_atributos_ds1,fim_atributos_ds1))
ds1 = df.iloc[:,atts]

atts = [2] + list(range(fim_atributos_ds1,fim_atributos_ds1+n_atibutos))
ds2 = df.iloc[:,atts]

ds1.columns = colunas
ds2.columns = colunas

In [123]:
#nan em coluna de interiro
ds2.zipcode = ds2.zipcode.fillna(-1)
ds2.zipcode = ds2.zipcode.astype(int)
ds2.zipcode = ds2.zipcode.astype(str)
ds2.zipcode = ds2.zipcode.replace('-1', np.nan)

ds1.zipcode = ds1.zipcode.fillna(-1)
ds1.zipcode = ds1.zipcode.astype(int)
ds1.zipcode = ds1.zipcode.astype(str)
ds1.zipcode = ds1.zipcode.replace('-1', np.nan)


D:\Programas\Anaconda3\envs\utils\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [126]:
ds1 = retornar_ds_limpo(ds1)
ds2 = retornar_ds_limpo(ds2)

#### Estatisticas

In [127]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

,address,city,name,phone,state,zipcode,ds
metrica,,,,,,,
count,323,315,315,315,315,315,fodors
unique,233,44,236,241,10,63,fodors
top,1360 El Camino Real,Menlo Park,Iron Gate Restaurant,(650) 592-7893,CA,94025,fodors
freq,13,48,13,13,226,48,fodors
nan,0,0,8,8,8,8,fodors


,address,city,name,phone,state,zipcode,ds
metrica,,,,,,,
count,238,238,238,237,238,238,zagats
unique,209,47,219,220,9,61,zagats
top,1 S County Rd,Menlo Park,Pizza My Heart,(858) 792-8466,CA,94025,zagats
freq,5,36,2,2,151,36,zagats
nan,0,8,8,8,9,8,zagats


#### Limpando os dados

Sem valores nulos

In [128]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2
0,20,71
8,80,71
52,685,1211
53,690,1742
107,1123,738


In [129]:
remover_da_lista2 = []
remover_da_lista1 = []

for i in da.id1.unique():
    remover_da_lista2.append(da[da.id1 == i].id2.values[-1])

for i in db.id2.unique():
    remover_da_lista1.append(db[db.id2 == i].id1.values[-1])

print(remover_da_lista1)
print(remover_da_lista2)

['80', '1641', '1646', '1609', '1364', '1482', '2257', '2194', '3380', '4843']
[]


In [130]:
cds1 = ds1[~ds1.id.isin(remover_da_lista1)]
cds2 = ds2[~ds2.id.isin(remover_da_lista2)]

gs = gs[~gs.id1.isin(remover_da_lista1)]
gs = gs[~gs.id2.isin(remover_da_lista2)]

ds1 = cds1
ds2 = cds2

In [120]:
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2


Dataset clean-clean

In [134]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)

### restaurantes1

In [147]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Magellan' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'restaurants'+os.sep

inputZip = 'yelp_zomato_big.zip'
data = 'labeled_data'


zip_file = 'processed_yelp-zomato.zip'

df = read(ds_files+inputZip,data,skip_coments=True)

In [148]:
# gabarito
gs = df[df.gold==1].iloc[:,[1,2]]

colunas =  [ 'id','NAME','PHONENUMBER','ADDRESS' ]
n_atibutos = len(colunas) -1

inicio_atributos_ds1 = 3
fim_atributos_ds1 = inicio_atributos_ds1 + n_atibutos


atts = [1] + list(range(inicio_atributos_ds1,fim_atributos_ds1))
ds1 = df.iloc[:,atts]

atts = [2] + list(range(fim_atributos_ds1,fim_atributos_ds1+n_atibutos))
ds2 = df.iloc[:,atts]

ds1.columns = colunas
ds2.columns = colunas

In [149]:
ds1 = retornar_ds_limpo(ds1)
ds2 = retornar_ds_limpo(ds2)

#### Estatisticas

In [150]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

,NAME,PHONENUMBER,ADDRESS,ds
metrica,,,,
count,304,303,303,fodors
unique,302,302,299,fodors
top,Harold's Chicken Shack,(608) 819-6319,"100 State St, Madison, WI",fodors
freq,2,2,2,fodors
nan,0,0,1,fodors


,NAME,PHONENUMBER,ADDRESS,ds
metrica,,,,
count,324,324,324,zagats
unique,305,303,307,zagats
top,Lao Laan,1445980000000,"1146 Williamson St, Madison, WI 53703",zagats
freq,5,8,3,zagats
nan,0,3,3,zagats


#### Limpando os dados

Sem valores nulos

In [151]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2
45,334,641
46,334,642
277,1972,3764
278,1972,3765
372,2550,4878


,id1,id2


In [152]:
remover_da_lista2 = []
remover_da_lista1 = []

for i in da.id1.unique():
    remover_da_lista2.append(da[da.id1 == i].id2.values[-1])

for i in db.id2.unique():
    remover_da_lista1.append(db[db.id2 == i].id1.values[-1])

print(remover_da_lista1)
print(remover_da_lista2)

[]
['642', '3765', '4879', '4891', '5500']


In [153]:
cds1 = ds1[~ds1.id.isin(remover_da_lista1)]
cds2 = ds2[~ds2.id.isin(remover_da_lista2)]

gs = gs[~gs.id1.isin(remover_da_lista1)]
gs = gs[~gs.id2.isin(remover_da_lista2)]

ds1 = cds1
ds2 = cds2

In [154]:
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2


Dataset clean-clean

In [155]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)

## Books

### Books1

In [187]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Magellan' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'books'+os.sep

inputZip = 'books1.zip'
data = 'labeled_data'


zip_file = 'processed_amazon-barnesnobel-small.zip'

df = read(ds_files+inputZip,data,skip_coments=True)

b'Skipping line 6: expected 1 fields, saw 10\nSkipping line 7: expected 1 fields, saw 10\nSkipping line 8: expected 1 fields, saw 10\nSkipping line 9: expected 1 fields, saw 10\nSkipping line 10: expected 1 fields, saw 10\nSkipping line 11: expected 1 fields, saw 10\nSkipping line 12: expected 1 fields, saw 10\nSkipping line 13: expected 1 fields, saw 10\nSkipping line 14: expected 1 fields, saw 10\nSkipping line 15: expected 1 fields, saw 10\nSkipping line 16: expected 1 fields, saw 10\nSkipping line 17: expected 1 fields, saw 10\nSkipping line 18: expected 1 fields, saw 10\nSkipping line 19: expected 1 fields, saw 10\nSkipping line 20: expected 1 fields, saw 10\nSkipping line 21: expected 1 fields, saw 10\nSkipping line 22: expected 1 fields, saw 10\nSkipping line 23: expected 1 fields, saw 10\nSkipping line 24: expected 1 fields, saw 10\nSkipping line 25: expected 1 fields, saw 10\nSkipping line 26: expected 1 fields, saw 10\nSkipping line 27: expected 1 fields, saw 10\nSkipping lin

In [188]:
# gabarito
gs = df[df.gold_label==1].iloc[:,[1,2]]

colunas =  [ 'id','title','pubyear','pages' ]
n_atibutos = len(colunas) -1

inicio_atributos_ds1 = 3
fim_atributos_ds1 = inicio_atributos_ds1 + n_atibutos


atts = [1] + list(range(inicio_atributos_ds1,fim_atributos_ds1))
ds1 = df.iloc[:,atts]

atts = [2] + list(range(fim_atributos_ds1,fim_atributos_ds1+n_atibutos))
ds2 = df.iloc[:,atts]

ds1.columns = colunas
ds2.columns = colunas

In [189]:
ds1.pages = ds1.pages.fillna(-1)
ds1.pages = ds1.pages.astype(int)
ds1.pages = ds1.pages.astype(str)
ds1.pages = ds1.pages.replace('-1', np.nan)

ds2.pages = ds2.pages.fillna(-1)
ds2.pages = ds2.pages.astype(int)
ds2.pages = ds2.pages.astype(str)
ds2.pages = ds2.pages.replace('-1', np.nan)


D:\Programas\Anaconda3\envs\utils\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [190]:
ds1 = retornar_ds_limpo(ds1)
ds2 = retornar_ds_limpo(ds2)

#### Estatisticas

In [191]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

,title,pages,ds
metrica,,,
count,360,341,fodors
unique,353,213,fodors
top,Data Warehousing,320,fodors
freq,4,8,fodors
nan,0,0,fodors


,title,pages,ds
metrica,,,
count,351,341,zagats
unique,349,225,zagats
top,Microsoft Access 2000,400,zagats
freq,2,7,zagats
nan,0,3,zagats


#### Limpando os dados

Sem valores nulos

In [192]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2


Dataset clean-clean

In [193]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)

### Books2

In [176]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Magellan' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'books'+os.sep

inputZip = 'books2.zip'
data = 'labeled_data'


zip_file = 'processed_goodreads-barnesnobel.zip'

df = read(ds_files+inputZip,data,skip_coments=True)

b'Skipping line 6: expected 1 fields, saw 21\nSkipping line 7: expected 1 fields, saw 21\nSkipping line 8: expected 1 fields, saw 21\nSkipping line 9: expected 1 fields, saw 21\nSkipping line 10: expected 1 fields, saw 21\nSkipping line 11: expected 1 fields, saw 21\nSkipping line 12: expected 1 fields, saw 21\nSkipping line 13: expected 1 fields, saw 21\nSkipping line 14: expected 1 fields, saw 21\nSkipping line 15: expected 1 fields, saw 21\nSkipping line 16: expected 1 fields, saw 21\nSkipping line 17: expected 1 fields, saw 21\nSkipping line 18: expected 1 fields, saw 21\nSkipping line 19: expected 1 fields, saw 21\nSkipping line 20: expected 1 fields, saw 21\nSkipping line 21: expected 1 fields, saw 21\nSkipping line 22: expected 1 fields, saw 21\nSkipping line 23: expected 1 fields, saw 21\nSkipping line 24: expected 1 fields, saw 21\nSkipping line 25: expected 1 fields, saw 21\nSkipping line 26: expected 1 fields, saw 21\nSkipping line 27: expected 1 fields, saw 21\nSkipping lin

In [177]:
df.columns

Index(['_id', 'ltable.ID', 'rtable.ID', 'ltable.FirstAuthor', 'ltable.ISBN',
       'ltable.ISBN13', 'ltable.PageCount', 'ltable.PublishDate',
       'ltable.Publisher', 'ltable.SecondAuthor', 'ltable.ThirdAuthor',
       'ltable.Title', 'rtable.FirstAuthor', 'rtable.SecondAuthor',
       'rtable.ThirdAuthor', 'rtable.ISBN13', 'rtable.PageCount',
       'rtable.PublishDate', 'rtable.Publisher', 'rtable.Title',
       'match_label'],
      dtype='object')

In [178]:
# gabarito
gs = df[df.match_label==1].iloc[:,[1,2]]

colunas_1 =  [ 'id', 'FirstAuthor', 'ISBN',
       'ISBN13', 'PageCount', 'PublishDate',
       'Publisher', 'SecondAuthor', 'ThirdAuthor',
       'Title' ]

colunas_2 =  [ 'id', 'FirstAuthor', 'SecondAuthor',
       'ThirdAuthor', 'ISBN13', 'PageCount',
       'PublishDate', 'Publisher', 'Title' ]

n_atibutos = len(colunas_1) -1

inicio_atributos_ds1 = 3
fim_atributos_ds1 = inicio_atributos_ds1 + n_atibutos


atts = [1] + list(range(inicio_atributos_ds1,fim_atributos_ds1))
ds1 = df.iloc[:,atts]

n_atibutos = len(colunas_2) -1
atts = [2] + list(range(fim_atributos_ds1,fim_atributos_ds1+n_atibutos))
ds2 = df.iloc[:,atts]

ds1.columns = colunas_1
ds2.columns = colunas_2

In [179]:
ds1 = retornar_ds_limpo(ds1)
ds2 = retornar_ds_limpo(ds2)

#### Estatisticas

In [180]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

,FirstAuthor,ISBN,PublishDate,Publisher,SecondAuthor,ThirdAuthor,Title,ds
metrica,,,,,,,,
count,286,227,244,258,77,23,286,fodors
unique,223,223,228,196,74,22,275,fodors
top,Mark Twain,8132016939,September 1st 2011,Nabu Press,Harriet E. Smith,Grover Gardner,Autobiography,fodors
freq,17,2,2,6,3,2,4,fodors
nan,0,0,59,63,0,42,28,fodors


,FirstAuthor,SecondAuthor,ThirdAuthor,PublishDate,Publisher,Title,ds
metrica,,,,,,,
count,317,41,7,270,317,317,zagats
unique,227,40,7,249,211,311,zagats
top,Benjamin Franklin,Grover Gardner,Published By Northpointe Classics,1/28/13,CreateSpace Publishing,An Autobiography Volume 2,zagats
freq,28,2,1,4,12,2,zagats
nan,0,10,286,320,10,107,zagats


#### Limpando os dados

Sem valores nulos

In [181]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2
6,113,63
239,2473,63
244,2519,2981
253,2704,2981


In [182]:
remover_da_lista2 = []
remover_da_lista1 = []

for i in da.id1.unique():
    remover_da_lista2.append(da[da.id1 == i].id2.values[-1])

for i in db.id2.unique():
    remover_da_lista1.append(db[db.id2 == i].id1.values[-1])

print(remover_da_lista1)
print(remover_da_lista2)

['2473', '2704']
[]


In [183]:
cds1 = ds1[~ds1.id.isin(remover_da_lista1)]
cds2 = ds2[~ds2.id.isin(remover_da_lista2)]

gs = gs[~gs.id1.isin(remover_da_lista1)]
gs = gs[~gs.id2.isin(remover_da_lista2)]

ds1 = cds1
ds2 = cds2

In [184]:
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2


Dataset clean-clean

In [185]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)

### Books4

In [201]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Magellan' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'restaurants'+os.sep

inputZip = 'books4.zip'
data = 'labeled_data'


zip_file = 'processed_amazon-barnesnobel.zip'

df = read(ds_files+inputZip,data,skip_coments=True)

b'Skipping line 6: expected 1 fields, saw 12\nSkipping line 7: expected 1 fields, saw 12\nSkipping line 8: expected 1 fields, saw 12\nSkipping line 9: expected 1 fields, saw 12\nSkipping line 10: expected 1 fields, saw 12\nSkipping line 11: expected 1 fields, saw 12\nSkipping line 12: expected 1 fields, saw 12\nSkipping line 13: expected 1 fields, saw 12\nSkipping line 14: expected 1 fields, saw 12\nSkipping line 15: expected 1 fields, saw 12\nSkipping line 16: expected 1 fields, saw 12\nSkipping line 17: expected 1 fields, saw 12\nSkipping line 18: expected 1 fields, saw 12\nSkipping line 19: expected 1 fields, saw 12\nSkipping line 20: expected 1 fields, saw 12\nSkipping line 21: expected 1 fields, saw 12\nSkipping line 22: expected 1 fields, saw 12\nSkipping line 23: expected 1 fields, saw 12\nSkipping line 24: expected 1 fields, saw 12\nSkipping line 25: expected 1 fields, saw 12\nSkipping line 26: expected 1 fields, saw 12\nSkipping line 27: expected 1 fields, saw 12\nSkipping lin

In [202]:
df.columns

Index(['_id', 'ltable.ID', 'rtable.ID', 'ltable.Title', 'ltable.Author',
       'ltable.ISBN_13', 'ltable.Publisher', 'rtable.Title', 'rtable.Author',
       'rtable.ISBN_13', 'rtable.Publisher', 'gold'],
      dtype='object')

In [203]:
# gabarito
gs = df[df.gold==1].iloc[:,[1,2]]

colunas_1 =  [ 'id', 'Title', 'Author', 'ISBN_13', 'Publisher' ]

colunas_2 =  colunas_1

n_atibutos = len(colunas_1) -1

inicio_atributos_ds1 = 3
fim_atributos_ds1 = inicio_atributos_ds1 + n_atibutos


atts = [1] + list(range(inicio_atributos_ds1,fim_atributos_ds1))
ds1 = df.iloc[:,atts]

n_atibutos = len(colunas_2) -1
atts = [2] + list(range(fim_atributos_ds1,fim_atributos_ds1+n_atibutos))
ds2 = df.iloc[:,atts]

ds1.columns = colunas_1
ds2.columns = colunas_2

In [204]:
ds1 = retornar_ds_limpo(ds1)
ds2 = retornar_ds_limpo(ds2)

#### Estatisticas

In [205]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

,Title,Author,ISBN_13,Publisher,ds
metrica,,,,,
count,342,341,143,162,fodors
unique,334,237,143,48,fodors
top,iPod: The Missing Manual,David D. Busch,978-0596005818,O'Reilly Media,fodors
freq,3,13,1,21,fodors
nan,0,0,1,199,fodors


,Title,Author,ISBN_13,Publisher,ds
metrica,,,,,
count,244,240,199,199,zagats
unique,242,157,199,47,zagats
top,C,David D. Busch,978-1285843285,Cengage Learning,zagats
freq,2,23,1,38,zagats
nan,0,144,148,189,zagats


#### Limpando os dados

Sem valores nulos

In [206]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2
140,1996,1008
148,2058,1008


In [207]:
remover_da_lista2 = []
remover_da_lista1 = []

for i in da.id1.unique():
    remover_da_lista2.append(da[da.id1 == i].id2.values[-1])

for i in db.id2.unique():
    remover_da_lista1.append(db[db.id2 == i].id1.values[-1])

print(remover_da_lista1)
print(remover_da_lista2)

['2058']
[]


In [208]:
cds1 = ds1[~ds1.id.isin(remover_da_lista1)]
cds2 = ds2[~ds2.id.isin(remover_da_lista2)]

gs = gs[~gs.id1.isin(remover_da_lista1)]
gs = gs[~gs.id2.isin(remover_da_lista2)]

ds1 = cds1
ds2 = cds2

In [209]:
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2


Dataset clean-clean

In [210]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)

## Movies

### Movies1

In [238]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Magellan' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'movies'+os.sep

inputZip = 'movies1.zip'
data = 'labeled_data'


zip_file = 'processed_imdb-rottentomatos.zip'

df = read(ds_files+inputZip,data,skip_coments=True)

b'Skipping line 6: expected 1 fields, saw 10\nSkipping line 7: expected 1 fields, saw 10\nSkipping line 8: expected 1 fields, saw 10\nSkipping line 9: expected 1 fields, saw 10\nSkipping line 10: expected 1 fields, saw 10\nSkipping line 11: expected 1 fields, saw 10\nSkipping line 12: expected 1 fields, saw 10\nSkipping line 13: expected 1 fields, saw 10\nSkipping line 14: expected 1 fields, saw 10\nSkipping line 15: expected 1 fields, saw 10\nSkipping line 16: expected 1 fields, saw 10\nSkipping line 17: expected 1 fields, saw 10\nSkipping line 18: expected 1 fields, saw 10\nSkipping line 19: expected 1 fields, saw 10\nSkipping line 20: expected 1 fields, saw 10\nSkipping line 21: expected 1 fields, saw 10\nSkipping line 22: expected 1 fields, saw 10\nSkipping line 23: expected 1 fields, saw 10\nSkipping line 24: expected 1 fields, saw 10\nSkipping line 25: expected 1 fields, saw 10\nSkipping line 26: expected 1 fields, saw 10\nSkipping line 27: expected 1 fields, saw 10\nSkipping lin

In [239]:
df.columns

Index(['_id', 'ltable.Id', 'rtable.Id', 'ltable.Director', 'ltable.Name',
       'ltable.Year', 'rtable.Director', 'rtable.Name', 'rtable.YearRange',
       'gold'],
      dtype='object')

In [240]:
# gabarito
gs = df[df.gold==1].iloc[:,[1,2]]

colunas_1 =  [ 'id', 'Director', 'Name', 'Year' ]

colunas_2 =  colunas_1

n_atibutos = len(colunas_1) -1

inicio_atributos_ds1 = 3
fim_atributos_ds1 = inicio_atributos_ds1 + n_atibutos


atts = [1] + list(range(inicio_atributos_ds1,fim_atributos_ds1))
ds1 = df.iloc[:,atts]

n_atibutos = len(colunas_2) -1
atts = [2] + list(range(fim_atributos_ds1,fim_atributos_ds1+n_atibutos))
ds2 = df.iloc[:,atts]

ds1.columns = colunas_1
ds2.columns = colunas_2

In [241]:
ds1.Year = ds1.Year.astype(str)
ds2.Year = ds2.Year.astype(str)

D:\Programas\Anaconda3\envs\utils\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [242]:
ds1 = retornar_ds_limpo(ds1)
ds2 = retornar_ds_limpo(ds2)

#### Estatisticas

In [243]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

,id,Director,Name,Year,ds
metrica,,,,,
count,558,558,558,558,fodors
unique,558,476,554,78,fodors
top,tt0046269,Roger Corman,Beauty and the Beast,2013,fodors
freq,1,5,2,42,fodors
nan,0,0,0,0,fodors


,id,Director,Name,Year,ds
metrica,,,,,
count,556,556,556,556,zagats
unique,556,481,549,76,zagats
top,racing_extinction,Roger Corman,The Hive,2014 2015 2016,zagats
freq,1,6,2,47,zagats
nan,0,0,0,0,zagats


#### Limpando os dados

Sem valores nulos

In [244]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2


In [222]:
remover_da_lista2 = []
remover_da_lista1 = []

for i in da.id1.unique():
    remover_da_lista2.append(da[da.id1 == i].id2.values[-1])

for i in db.id2.unique():
    remover_da_lista1.append(db[db.id2 == i].id1.values[-1])

print(remover_da_lista1)
print(remover_da_lista2)

[]
[]


In [223]:
cds1 = ds1[~ds1.id.isin(remover_da_lista1)]
cds2 = ds2[~ds2.id.isin(remover_da_lista2)]

gs = gs[~gs.id1.isin(remover_da_lista1)]
gs = gs[~gs.id2.isin(remover_da_lista2)]

ds1 = cds1
ds2 = cds2

In [224]:
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2


Dataset clean-clean

In [245]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)

### Movies2

In [246]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Magellan' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'movies'+os.sep

inputZip = 'movies2.zip'
data = 'labeled_data'


zip_file = 'processed_imdb-tmd.zip'

df = read(ds_files+inputZip,data,skip_coments=True)

b'Skipping line 6: expected 1 fields, saw 8\nSkipping line 7: expected 1 fields, saw 8\nSkipping line 8: expected 1 fields, saw 8\nSkipping line 9: expected 1 fields, saw 8\nSkipping line 10: expected 1 fields, saw 8\nSkipping line 11: expected 1 fields, saw 8\nSkipping line 12: expected 1 fields, saw 8\nSkipping line 13: expected 1 fields, saw 8\nSkipping line 14: expected 1 fields, saw 8\nSkipping line 15: expected 1 fields, saw 8\nSkipping line 16: expected 1 fields, saw 8\nSkipping line 17: expected 1 fields, saw 8\nSkipping line 18: expected 1 fields, saw 8\nSkipping line 19: expected 1 fields, saw 8\nSkipping line 20: expected 1 fields, saw 8\nSkipping line 21: expected 1 fields, saw 8\nSkipping line 22: expected 1 fields, saw 8\nSkipping line 23: expected 1 fields, saw 8\nSkipping line 24: expected 1 fields, saw 8\nSkipping line 25: expected 1 fields, saw 8\nSkipping line 26: expected 1 fields, saw 8\nSkipping line 27: expected 1 fields, saw 8\nSkipping line 28: expected 1 field

In [247]:
df.columns

Index(['_id', 'ltable.ID', 'rtable.ID', 'ltable.title', 'ltable.year',
       'rtable.title', 'rtable.year', 'class_label'],
      dtype='object')

In [248]:
# gabarito
gs = df[df.class_label==1].iloc[:,[1,2]]

colunas_1 =  [ 'id', 'title', 'year' ]

colunas_2 =  colunas_1

n_atibutos = len(colunas_1) -1

inicio_atributos_ds1 = 3
fim_atributos_ds1 = inicio_atributos_ds1 + n_atibutos


atts = [1] + list(range(inicio_atributos_ds1,fim_atributos_ds1))
ds1 = df.iloc[:,atts]

n_atibutos = len(colunas_2) -1
atts = [2] + list(range(fim_atributos_ds1,fim_atributos_ds1+n_atibutos))
ds2 = df.iloc[:,atts]

ds1.columns = colunas_1
ds2.columns = colunas_2

In [218]:
ds2.Year = ds2.Year.astype(str)
ds2.Year = ds2.Year.astype(str)

D:\Programas\Anaconda3\envs\utils\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [251]:
ds1 = retornar_ds_limpo(ds1)
ds2 = retornar_ds_limpo(ds2)

#### Estatisticas

In [252]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

,title,ds
metrica,,
count,359,fodors
unique,359,fodors
top,End of Watch,fodors
freq,1,fodors
nan,0,fodors


,title,ds
metrica,,
count,354,zagats
unique,354,zagats
top,The Seeker: The Dark Is Rising,zagats
freq,1,zagats
nan,0,zagats


#### Limpando os dados

Sem valores nulos

In [253]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2


In [222]:
remover_da_lista2 = []
remover_da_lista1 = []

for i in da.id1.unique():
    remover_da_lista2.append(da[da.id1 == i].id2.values[-1])

for i in db.id2.unique():
    remover_da_lista1.append(db[db.id2 == i].id1.values[-1])

print(remover_da_lista1)
print(remover_da_lista2)

[]
[]


In [223]:
cds1 = ds1[~ds1.id.isin(remover_da_lista1)]
cds2 = ds2[~ds2.id.isin(remover_da_lista2)]

gs = gs[~gs.id1.isin(remover_da_lista1)]
gs = gs[~gs.id2.isin(remover_da_lista2)]

ds1 = cds1
ds2 = cds2

In [224]:
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2


Dataset clean-clean

In [254]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)

### Movies2

In [246]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Magellan' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'movies'+os.sep

inputZip = 'movies2.zip'
data = 'labeled_data'


zip_file = 'processed_imdb-tmd.zip'

df = read(ds_files+inputZip,data,skip_coments=True)

b'Skipping line 6: expected 1 fields, saw 8\nSkipping line 7: expected 1 fields, saw 8\nSkipping line 8: expected 1 fields, saw 8\nSkipping line 9: expected 1 fields, saw 8\nSkipping line 10: expected 1 fields, saw 8\nSkipping line 11: expected 1 fields, saw 8\nSkipping line 12: expected 1 fields, saw 8\nSkipping line 13: expected 1 fields, saw 8\nSkipping line 14: expected 1 fields, saw 8\nSkipping line 15: expected 1 fields, saw 8\nSkipping line 16: expected 1 fields, saw 8\nSkipping line 17: expected 1 fields, saw 8\nSkipping line 18: expected 1 fields, saw 8\nSkipping line 19: expected 1 fields, saw 8\nSkipping line 20: expected 1 fields, saw 8\nSkipping line 21: expected 1 fields, saw 8\nSkipping line 22: expected 1 fields, saw 8\nSkipping line 23: expected 1 fields, saw 8\nSkipping line 24: expected 1 fields, saw 8\nSkipping line 25: expected 1 fields, saw 8\nSkipping line 26: expected 1 fields, saw 8\nSkipping line 27: expected 1 fields, saw 8\nSkipping line 28: expected 1 field

In [247]:
df.columns

Index(['_id', 'ltable.ID', 'rtable.ID', 'ltable.title', 'ltable.year',
       'rtable.title', 'rtable.year', 'class_label'],
      dtype='object')

In [248]:
# gabarito
gs = df[df.class_label==1].iloc[:,[1,2]]

colunas_1 =  [ 'id', 'title', 'year' ]

colunas_2 =  colunas_1

n_atibutos = len(colunas_1) -1

inicio_atributos_ds1 = 3
fim_atributos_ds1 = inicio_atributos_ds1 + n_atibutos


atts = [1] + list(range(inicio_atributos_ds1,fim_atributos_ds1))
ds1 = df.iloc[:,atts]

n_atibutos = len(colunas_2) -1
atts = [2] + list(range(fim_atributos_ds1,fim_atributos_ds1+n_atibutos))
ds2 = df.iloc[:,atts]

ds1.columns = colunas_1
ds2.columns = colunas_2

In [218]:
ds2.Year = ds2.Year.astype(str)
ds2.Year = ds2.Year.astype(str)

D:\Programas\Anaconda3\envs\utils\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [251]:
ds1 = retornar_ds_limpo(ds1)
ds2 = retornar_ds_limpo(ds2)

#### Estatisticas

In [252]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

,title,ds
metrica,,
count,359,fodors
unique,359,fodors
top,End of Watch,fodors
freq,1,fodors
nan,0,fodors


,title,ds
metrica,,
count,354,zagats
unique,354,zagats
top,The Seeker: The Dark Is Rising,zagats
freq,1,zagats
nan,0,zagats


#### Limpando os dados

Sem valores nulos

In [253]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2


In [222]:
remover_da_lista2 = []
remover_da_lista1 = []

for i in da.id1.unique():
    remover_da_lista2.append(da[da.id1 == i].id2.values[-1])

for i in db.id2.unique():
    remover_da_lista1.append(db[db.id2 == i].id1.values[-1])

print(remover_da_lista1)
print(remover_da_lista2)

[]
[]


In [223]:
cds1 = ds1[~ds1.id.isin(remover_da_lista1)]
cds2 = ds2[~ds2.id.isin(remover_da_lista2)]

gs = gs[~gs.id1.isin(remover_da_lista1)]
gs = gs[~gs.id2.isin(remover_da_lista2)]

ds1 = cds1
ds2 = cds2

In [224]:
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2


Dataset clean-clean

In [254]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)

# Corleone

## restaurants fodors-zagats

In [21]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Corleone' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'restaurants'+os.sep

inputZip = 'restaurants.zip'
ds1_name = 'fodors'
ds2_name = 'zagats'
gs_name ='matches_fodors_zagats'


zip_file = 'processed_fodors-zagats.zip'

ds1 = read(ds_files+inputZip,ds1_name,quote="'")
ds2 = read(ds_files+inputZip,ds2_name,quote="'")
gs = read(ds_files+inputZip,gs_name)

### Limpando

In [25]:
# ds1.pop('title')
# z = ds2.pop('title')
# del z

ds1.name = ds1.name.astype(str)
ds1.addr = ds1.addr.astype(str)
ds1.city = ds1.city.astype(str)
ds1.phone = ds1.phone.astype(str)

ds2.name = ds2.name.astype(str)
ds2.addr = ds2.addr.astype(str)
ds2.city = ds2.city.astype(str)
ds2.phone = ds2.phone.astype(str)

### Estatisticas

In [26]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

,name,addr,city,phone,type,ds
metrica,,,,,,
count,533,533,533,533,532,fodors
unique,528,519,17,528,40,fodors
top,morton\s',3570 las vegas blvd. s,new york,702/731-7547,american,fodors
freq,2,5,250,2,137,fodors
nan,0,0,0,0,0,fodors


,name,addr,city,phone,type,ds
metrica,,,,,,
count,331,331,331,331,331,zagats
unique,331,322,46,329,58,zagats
top,piero\s restaurant',3799 las vegas blvd. s.,new york city,404-659-0400,american (new),zagats
freq,1,3,88,2,25,zagats
nan,0,0,0,0,0,zagats


### Limpando os dados

Sem valores nulos

In [27]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2


Dataset clean-clean

### Salvando os dados

In [28]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)